In [ ]:
import os
import pandas as pd
import io
import unicodedata

root_dir = 'coral_and_sponges_all/data/pub/data/paleo/coral'


In [ ]:
root_files = os.listdir(root_dir)
root_files = root_files[0:3] + root_files[4:6] + root_files[9:12]
sub_files_to_flatten = [[f'{r}/{f}' for f in os.listdir(f'{root_dir}/{r}')] for r in root_files]
#Flatten list of files into one list
sub_files = [f'{root_dir}/{item}' for sublist in sub_files_to_flatten for item in sublist]

In [ ]:
len(sub_files)

In [ ]:
def create_coral_file_df(filename):
    try:
        # Using a weird encoding to solve file read issue; see https://stackoverflow.com/questions/46000191/utf-8-codec-cant-decode-byte-0x92-in-position-18-invalid-start-byte
        coral_file = open(filename,'r', encoding="cp1252")
        file_in = coral_file.readlines()

        for i in range(len(file_in)):
            line = file_in[i]
            if "Location:" in line:
                location = line.split(": ")[1]
            if "Northernmost_Latitude:" in line:
                northernmost_lat = line.split(": ")[1]
            if "Southernmost_Latitude:" in line:
                southernmost_lat = line.split(": ")[1]
            if "Easternmost_Longitude:" in line:
                easternmost_lat = line.split(": ")[1]
            if "Westernmost_Longitude:" in line:
                westernmost_lat = line.split(": ")[1]
            if "Elevation:" in line:
                elevation = line.split(": ")[1].split(' ')[0]
            if "Earliest_Year:" in line:
                earliest_year = line.split(": ")[1]
            if "Most_Recent_Year:" in line:
                most_recent_year = line.split(": ")[1]
            if "Species_Name:" in line:
                species = line.split(": ")[1].strip('\n')
            if not line.startswith('#'):
                print(line)
                age_line = i
                extracted = ' '.join(file_in[age_line:])
                # print(extracted)

        try:
            lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
            lon = (float(easternmost_lat) + float(westernmost_lat)) / 2

            data = io.StringIO(extracted)
            df = pd.read_csv(data, sep="\t", error_bad_lines=False)
            df = df[['age', 'd18O']]
            df = df[df['d18O'] != 'NAN']
            df['lat'] = lat
            df['lon'] = lon
            df['elevation'] = elevation
            df['species'] = species
            return(df)

        except Exception as e:
            print("ERROR : "+str(e))
            # print("Error on line {}".format(sys.exc_info()[-1].tb_lineno))
            # print('hi')
    except Exception as e:
            print("ERROR : "+str(e))

Quick Proof of Concept for the function above:

In [ ]:
filename = f'{root_dir}/atlantic/barbados_coral18o.txt'
#"coral_and_sponges_all/atlantic/goodkin2008-sst_noaa.txt"
#'swart1998-st-1_noaa.txt'

create_coral_file_df(filename)

In [ ]:
all_coral_df = pd.DataFrame(columns=['age', 'd18O', 'lat', 'lon', 'elevation', 'species'])
for file in sub_files:
    print(file)
    coral_df = create_coral_file_df(file)
    all_coral_df = pd.concat([all_coral_df, coral_df])

all_coral_df.reset_index
all_coral_df


In [ ]:
all_coral_df.to_csv('all_coral.csv')


In [ ]:
all_coral_df

In [ ]:
coral_file = 'swart1998-st-1_noaa.txt'
# Example of file with -noaa extension
file1 = open(coral_file,'r')
print(file1.read())
file1.close()

Gather metadata info! (site lat/lon & years of study)

In [ ]:
file1 = open(coral_file,'r')
file_in = file1.readlines()

for line in file_in:
    if "Location:" in line:
        location = line.split(": ")[1]
    if "Northernmost_Latitude:" in line:
        northernmost_lat = line.split(": ")[1]
    if "Southernmost_Latitude:" in line:
        southernmost_lat = line.split(": ")[1]
    if "Easternmost_Longitude:" in line:
        easternmost_lat = line.split(": ")[1]
    if "Westernmost_Longitude:" in line:
        westernmost_lat = line.split(": ")[1]
    if "Elevation:" in line:
        elevation = line.split(": ")[1].split(' ')[0]
    if "Earliest_Year:" in line:
        earliest_year = line.split(": ")[1]
    if "Most_Recent_Year:" in line:
        most_recent_year = line.split(": ")[1]
    if "Species_Name:" in line:
        species = line.split(": ")[1].strip('\n')

lat = (float(northernmost_lat) + float(southernmost_lat)) / 2
lon = (float(easternmost_lat) + float(westernmost_lat)) / 2



In [ ]:
age_line = 0
file1 = open(coral_file,'r')
with file1 as f:
    lines = f.readlines()
    for i in range(len(lines)):
        if lines[i].startswith('age'):
            print(i)
            age_line = i
            break
    extracted = ' '.join(lines[age_line:])
    print(extracted)

In [ ]:
data = io.StringIO(extracted)
df_test = pd.read_csv(data, sep="\t")
df_test.head()

Just grab `age` and `d18O` columns! And remove NAN values. 

In [ ]:
df_test_d18O = df_test[['age', 'd18O']]
df_test_d18O = df_test_d18O[df_test_d18O['d18O'] != 'NAN']
df_test_d18O['lat'] = lat
df_test_d18O['lon'] = lon
df_test_d18O['elevation'] = elevation
df_test_d18O['species'] = species
df_test_d18O